In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification

In [2]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
class SentimentClassifier(nn.Module):
    def __init__(self, model):
        super(SentimentClassifier, self).__init__()
        self.bert = model
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, 3)  # 768 - размерность выходного слоя BERT
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return self.softmax(logits)

# Подготовка данных
class SentimentDataset(Dataset):
    def __init__(self, reviews, labels):
        self.reviews = reviews
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        label = self.labels[idx]
        encoding = tokenizer.encode_plus(review,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt')
        return {
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labels': torch.tensor(label, dtype=torch.long)
    }


In [14]:
import pandas as pd

train = pd.read_csv("/home/jupyter/mnt/s3/hackathondata/train.csv")

reviews = train["sentence"] # список текстовых отзывов
labels = train["sentiment"].map({"+": 2, "?": 1, "−": 0}) # список меток классов (0 - негативный, 1 - нейтральный, 2 - положительный)
train_data = SentimentDataset(reviews[:800], labels[:800])
test_data = SentimentDataset(reviews[800:], labels[800:])

In [15]:
classifier = SentimentClassifier(model)
optimizer = optim.Adam(classifier.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

for epoch in range(10):
    running_loss = 0.0
    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        optimizer.zero_grad()
        outputs = classifier(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
print('Epoch %d loss: %.3f' % (epoch + 1, running_loss / len(train_loader)))

IndexError: tuple index out of range

In [9]:
train_loader

In [11]:
reviews.isna().any()

False

In [13]:
labels

0        2.0
1        NaN
2        NaN
3        NaN
4        2.0
        ... 
19356    NaN
19357    NaN
19358    2.0
19359    2.0
19360    NaN
Name: sentiment, Length: 19361, dtype: float64